In [21]:
import numpy as np
import cv2
from IPython.display import clear_output, Image, display, HTML
import ipywidgets as widgets
import threading
import numpy as np
import matplotlib.pyplot as plt

# Repeatability
np.random.seed(0)

In [22]:
#filename = "walking.mp4"
filename = "Trafic.mp4"
frame_height = 720
frame_width = 1280

In [23]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(filename)

fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"{fps} frames per second")

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()

  if ret == True:
    # Display the resulting frame
    cv2.imshow('Frame',frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(30) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

10 frames per second


In [24]:
def get_frames(filename):
    video = cv2.VideoCapture(filename)
    while video.isOpened():
        ret, frame = video.read()
        if ret:
            yield frame 
        else:
            break
    video.release()        
    yield None

In [25]:
def display(frame, particles, location):
    if len(particles)> 0:
        for i in range(NUM_PARTICLES):
            x = int(particles[i,0])
            y = int(particles[i,1])
            cv2.circle(frame, (x,y), 1 , (0,255,0) ,3)
    
    if len(location) > 0:
        cv2.circle(frame, location,15,(0,0,255),5)
    
    cv2.imshow('frame', frame)
    #stop the video if pressing q button
    if cv2.waitKey(30) & 0xFF == ord('q'):
        return True 

    return False

In [26]:
NUM_PARTICLES = 500
VEL_RANGE = 0.5

def initialize_particles():
    particles = np.random.rand(NUM_PARTICLES,4)
    particles = particles * np.array((frame_width,frame_height, VEL_RANGE,VEL_RANGE))
    particles[:,2:4] -= VEL_RANGE/2.0
    return particles



In [27]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]

    return particles

In [28]:
# Display moving particles on video
location = []
particles = initialize_particles()


for frame in get_frames(filename):
    if frame is None: break
    particles = apply_velocity(particles)

    terminate = display(frame, particles, location)
    if terminate:
        break

cv2.destroyAllWindows()   

In [29]:
def enforce_edges(particles):
    for i in range(NUM_PARTICLES):
        particles[i,0] = max(0,min(frame_width-1, particles[i,0]))
        particles[i,1] = max(0,min(frame_height-1, particles[i,1]))
    return particles

In [42]:
#TARGET_COLOR = np.array((66,63, 105))
TARGET_COLOR = np.array((100,0, 100))

def compute_errors(particles, frame):
    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        x = int(particles[i,0])
        y= int(particles[i,1])
        pixel_color = frame[y, x, :]
        errors[i] = np.sum((TARGET_COLOR - pixel_color)**2)
    return errors
print(frame.shape)

(900, 1908, 3)


In [43]:
print(particles)
def compute_weights(errors):
    weights = np.max(errors) - errors
    
    weights[
        (particles[:,0]==0) |
        (particles[:,0]==frame_width-1) |
        (particles[:,1]==0) |
        (particles[:,1]==frame_height-1) ] = 0  
    
    weights = weights**32
    print(particles)
    return weights

[[ 1.24526120e+02  9.39954651e+01  2.24873131e-01 -4.99969359e-01]
 [ 1.23815534e+02  9.31325310e+01 -1.19005179e-01  3.37974241e-01]
 [ 1.28078761e+02  9.72007511e+01 -2.56429636e-01 -2.82273869e-01]
 ...
 [ 1.30162143e+02  9.61844642e+01  4.40509395e-01  2.77477075e-03]
 [ 1.30576604e+02  9.34513362e+01  1.84033247e-01  6.94641731e-02]
 [ 1.27319499e+02  9.77130497e+01 -1.46770630e-01  9.91608928e-02]]


In [44]:
def resample(particles, weights):
    probabilities = weights / np.sum(weights)
    index_numbers = np.random.choice(
        NUM_PARTICLES,
        size=NUM_PARTICLES,
        p=probabilities)
    particles = particles[index_numbers, :]
    
    x = np.mean(particles[:,0])
    y= np.mean(particles[:,1])
    return particles, [int(x), int(y)]


In [45]:
POS_SIGMA = 0.75
VEL_SIGMA = 0.1

def apply_noise(particles):
    noise= np.concatenate(
    (
        np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, POS_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1)),
        np.random.normal(0.0, VEL_SIGMA, (NUM_PARTICLES,1))
    
    ),
    axis=1)
    
    particles += noise
    return particles

In [46]:
particles = initialize_particles()

for frame in get_frames(filename):
    if frame is None: break

    particles = apply_velocity(particles)
    particles = enforce_edges(particles)
    errors = compute_errors(particles, frame)
    weights = compute_weights(errors)
    particles, location = resample(particles, weights)
    particles = apply_noise(particles)
    terminate = display(frame, particles, tuple(location))
    if terminate:
        break
cv2.destroyAllWindows()

[[ 5.77958828e+00  6.86174191e+02 -1.90146682e-01 -1.30944291e-01]
 [ 7.85628559e+02  2.87508817e+02  2.28251029e-01 -1.59015230e-04]
 [ 3.95748078e+01  2.87474325e+02 -1.90775423e-01 -5.24313683e-02]
 ...
 [ 7.63450210e+01  1.96102557e+02  1.04630648e-01  9.86304672e-02]
 [ 9.83291463e+02  3.50030566e+02 -1.09352183e-01  1.59921815e-01]
 [ 4.71522497e+02  1.20118133e+02  1.80482388e-01 -1.67951798e-01]]
[[ 1.02169733e+03  3.06594974e+02  9.57907502e-02  1.78325033e-01]
 [ 4.58582386e+02  3.61862507e+01 -1.36097143e-01  7.71987850e-02]
 [ 9.11168660e+02  1.78779675e+02 -6.47790222e-03 -2.40339819e-01]
 ...
 [ 4.83390571e+02  3.97278913e+02 -1.99956722e-01 -2.38250532e-02]
 [ 8.53178606e+02  2.47105294e+02  3.22739229e-01 -1.32604860e-01]
 [ 2.86606884e+02  6.55429174e+02 -1.07380543e-01 -3.76205523e-02]]
[[ 1.12431702e+03  6.62754101e+02  1.64172689e-01 -3.23938139e-03]
 [ 9.35052823e+02  3.25342226e+02  1.30715067e-01  1.77449955e-01]
 [ 2.06083221e+02  5.07275806e+02 -3.56896476e-01 

[[ 7.96274635e+02  4.35684708e+02  2.07952407e-02  3.51341055e-01]
 [ 7.95891516e+02  4.32083306e+02 -2.14968378e-01  9.09121985e-02]
 [ 7.93814667e+02  4.37193636e+02 -4.76516812e-01  8.71841662e-01]
 ...
 [ 7.98076050e+02  4.32831798e+02  7.04513357e-01  1.89163025e-01]
 [ 7.95235120e+02  4.34061346e+02 -1.94653651e-01  4.57259788e-01]
 [ 7.95484886e+02  4.37166667e+02 -1.66080278e-01  1.04122897e+00]]
[[ 7.95348036e+02  4.30060477e+02  4.04186954e-02  4.13970505e-02]
 [ 7.97077219e+02  4.38177922e+02 -2.46489686e-02  9.72610770e-01]
 [ 7.94950692e+02  4.34673932e+02 -1.53284786e-01  3.37997105e-01]
 ...
 [ 7.95619317e+02  4.33004373e+02 -8.50316897e-02  6.15099688e-01]
 [ 7.95895803e+02  4.37222106e+02 -4.14600484e-01  1.27855195e-01]
 [ 7.96794756e+02  4.35714128e+02 -1.73077167e-01  4.63760746e-01]]
[[ 7.97013143e+02  4.36114271e+02  1.33826024e-01  3.45190951e-01]
 [ 7.95049711e+02  4.30213333e+02 -7.04198630e-02  2.16212985e-02]
 [ 7.97607111e+02  4.31160376e+02  1.22735327e-01 

[[ 7.97517372e+02  4.35610124e+02  6.48779221e-01  9.41813962e-02]
 [ 7.96829806e+02  4.34142591e+02 -5.04646259e-01 -3.32366316e-01]
 [ 7.99184621e+02  4.36990597e+02  4.88080042e-01 -4.97395456e-02]
 ...
 [ 7.94986224e+02  4.36639483e+02 -6.90021553e-02  2.34738446e-01]
 [ 7.98206293e+02  4.34438333e+02 -3.34753803e-02  7.96109610e-02]
 [ 7.97483594e+02  4.37500684e+02 -1.66202750e-01 -8.79621510e-02]]
[[ 7.96741825e+02  4.34553747e+02  9.62329279e-02  2.79028633e-01]
 [ 7.98999544e+02  4.33992528e+02  2.14372598e-01  5.13667163e-01]
 [ 7.98648167e+02  4.34056017e+02  1.89340564e-01 -2.80074632e-01]
 ...
 [ 7.97565366e+02  4.37906839e+02  3.78144590e-01  8.59693106e-02]
 [ 7.96252765e+02  4.27587664e+02 -1.23564758e-02 -6.41530815e-01]
 [ 7.98581986e+02  4.31911436e+02 -8.97365161e-02 -2.80389365e-01]]
[[ 7.98031046e+02  4.37296636e+02  3.04607043e-02  6.27202018e-01]
 [ 7.97981803e+02  4.33871803e+02  3.87660553e-01  4.01651942e-01]
 [ 7.97842497e+02  4.38564515e+02 -3.96675602e-01 

[[ 7.95786929e+02  4.36135954e+02 -2.50475098e-02  7.77935194e-01]
 [ 7.93608477e+02  4.34178161e+02 -2.75939355e-02  1.19646576e-01]
 [ 7.93511861e+02  4.29522487e+02 -5.34581023e-02  2.06369612e-01]
 ...
 [ 7.97068470e+02  4.34179054e+02 -1.85674024e-01  2.45044645e-01]
 [ 7.97745897e+02  4.38560210e+02 -8.49140900e-03  5.04383646e-01]
 [ 7.97018474e+02  4.31840996e+02 -1.06698827e-01 -5.86742872e-01]]
[[ 7.94434875e+02  4.28783297e+02 -1.47642769e-02  4.26019518e-01]
 [ 7.95980933e+02  4.34661017e+02 -3.27435767e-01  5.53053834e-01]
 [ 7.94723484e+02  4.28237902e+02 -5.82437891e-01 -2.36265509e-01]
 ...
 [ 7.98312301e+02  4.34374553e+02 -3.56952154e-01  6.12331505e-03]
 [ 7.93709494e+02  4.27898826e+02 -3.45854417e-01 -1.12910100e-01]
 [ 7.95979578e+02  4.31924770e+02 -1.44504791e-01  2.67284844e-01]]
[[ 7.97691712e+02  4.37557060e+02 -3.21752410e-02  4.88716430e-01]
 [ 7.97413609e+02  4.31328665e+02 -6.20076880e-02 -1.43062811e-01]
 [ 7.96054959e+02  4.34170003e+02  1.24234929e-01 

[[ 7.98678606e+02  4.32426074e+02  1.74172282e-01 -3.69432083e-01]
 [ 7.98130555e+02  4.35015119e+02 -1.77103095e-01  2.14156806e-01]
 [ 7.99784579e+02  4.33907957e+02  2.19780517e-01  2.68299097e-02]
 ...
 [ 7.97931462e+02  4.35703529e+02 -1.67689498e-01  7.81802466e-02]
 [ 7.98657131e+02  4.35965760e+02  2.60853769e-01  5.82883665e-01]
 [ 7.96612788e+02  4.35681720e+02 -5.61092594e-01 -3.35099762e-01]]
[[ 7.96799745e+02  4.32231513e+02  1.62261515e-01 -3.21670231e-02]
 [ 7.98727879e+02  4.34274253e+02  9.54020297e-01  1.51061017e-01]
 [ 7.95449643e+02  4.34567883e+02 -4.48579630e-01  2.22572878e-01]
 ...
 [ 7.96843459e+02  4.33208035e+02 -2.66845993e-01 -2.58910625e-01]
 [ 7.96104361e+02  4.32337175e+02 -6.95696120e-01 -8.94132219e-01]
 [ 7.97535830e+02  4.36803252e+02  4.78232542e-01  1.38079259e-01]]
[[ 7.97387739e+02  4.34455860e+02 -1.02577725e-01 -8.71443714e-02]
 [ 7.99741186e+02  4.32906869e+02 -2.30795375e-01 -7.88605511e-02]
 [ 7.98961990e+02  4.36769519e+02 -1.45005120e-02 

...

[[ 7.95566677e+02  4.36254755e+02 -3.17471101e-01 -1.89444221e-01]
 [ 7.99773419e+02  4.32943075e+02  4.70517531e-01  2.23720238e-01]
 [ 7.96374355e+02  4.34539368e+02  2.13573229e-01  6.34827690e-02]
 ...
 [ 7.97157181e+02  4.33290953e+02 -4.47579531e-01  1.49760651e-01]
 [ 7.96045578e+02  4.36591834e+02  2.84302068e-01  5.90490122e-01]
 [ 7.95047481e+02  4.34671860e+02 -1.43838042e-02  4.08119376e-01]]
[[ 7.96211441e+02  4.35560820e+02  3.84428704e-01  3.37899980e-01]
 [ 7.94216502e+02  4.29370639e+02 -4.61714951e-01 -5.08851561e-01]
 [ 7.96178212e+02  4.31118703e+02  1.75577048e-02  1.49014216e-01]
 ...
 [ 7.95423828e+02  4.36746884e+02  2.56975310e-01  4.59936100e-01]
 [ 7.96182620e+02  4.32597059e+02 -2.90190963e-01 -1.19421672e-01]
 [ 7.95593153e+02  4.33771342e+02  4.95352391e-01 -3.65668078e-01]]
[[ 7.98030269e+02  4.36252978e+02  2.80218623e-01  4.66690599e-01]
 [ 7.95535994e+02  4.35129996e+02  8.87003754e-02 -3.63599189e-01]
 [ 7.95792640e+02  4.37795551e+02  2.00716467e-01 

[[ 8.00237607e+02  4.51141086e+02 -2.62908418e-02  3.93070568e-01]
 [ 7.98689875e+02  4.50548349e+02  2.12852193e-01 -2.28187906e-01]
 [ 7.99716782e+02  4.50758612e+02 -8.45949364e-02  6.28204337e-01]
 ...
 [ 7.97638540e+02  4.51847462e+02  3.16089280e-01  2.12050317e-01]
 [ 7.99180237e+02  4.48707392e+02 -1.02500578e-01  3.39670576e-01]
 [ 8.01713247e+02  4.50144754e+02  3.94546738e-01  4.32503367e-01]]
[[ 7.99703752e+02  4.31664867e+02  1.56594468e-01 -3.23110419e-01]
 [ 7.99611442e+02  4.51247518e+02  5.66200301e-01  4.29342538e-01]
 [ 7.98936252e+02  4.51733220e+02  2.78736619e-01  4.04377410e-01]
 ...
 [ 7.99110702e+02  4.49665169e+02 -4.97918911e-02 -4.17596522e-02]
 [ 7.99842320e+02  4.49785654e+02  2.61775277e-01 -1.20271087e-02]
 [ 8.00093021e+02  4.50176026e+02 -6.28417699e-03  5.94122046e-01]]
[[ 7.98126332e+02  4.50060832e+02  3.50947471e-02  4.00952883e-01]
 [ 7.99875486e+02  4.53213180e+02  4.01152687e-01  6.92191344e-01]
 [ 7.89730916e+02  4.47551913e+02 -2.16221307e-01 

[[ 7.96336538e+02  4.29261705e+02 -1.27285715e-01 -1.59041314e-01]
 [ 7.96058632e+02  4.29996717e+02  2.37348206e-01 -9.05917682e-02]
 [ 7.95734218e+02  4.33333930e+02 -3.03937783e-01 -2.03823807e-01]
 ...
 [ 7.94641970e+02  4.29429368e+02 -8.10363023e-01  1.89604902e-01]
 [ 7.97467788e+02  4.31165025e+02  6.06602667e-02 -1.06024697e-01]
 [ 7.96077999e+02  4.31706565e+02 -3.11724088e-01  1.15451993e-01]]
[[ 7.94373328e+02  4.29919114e+02 -2.82377451e-01 -3.23183205e-01]
 [ 7.95678496e+02  4.28632852e+02  6.33727313e-02 -3.89304163e-01]
 [ 7.95103050e+02  4.29183066e+02 -5.29700867e-01 -4.22730863e-01]
 ...
 [ 7.96307008e+02  4.29853298e+02  3.68446386e-01 -1.87507743e-01]
 [ 7.94983496e+02  4.32088457e+02  9.28244457e-04 -1.05644125e-01]
 [ 7.96971651e+02  4.32829412e+02 -3.59879121e-01  2.74259909e-01]]
[[ 7.93364571e+02  4.34474923e+02 -1.09013890e+00  8.55950697e-01]
 [ 7.95883748e+02  4.29242574e+02 -3.43229306e-01 -3.21495264e-01]
 [ 7.96276199e+02  4.31906676e+02  1.07399373e-01 

[[ 7.98089624e+02  4.33124883e+02 -2.12865999e-02 -4.39149280e-01]
 [ 7.97177262e+02  4.34479477e+02 -5.70245953e-01 -1.31242508e-01]
 [ 7.97637020e+02  4.33166036e+02 -2.27402482e-01 -1.27498083e-01]
 ...
 [ 7.97003303e+02  4.33214073e+02  1.95284078e-01 -1.17230660e-01]
 [ 7.98912561e+02  4.34991613e+02 -2.45175252e-01  1.17133473e-02]
 [ 7.97770782e+02  4.35918095e+02 -3.30467353e-01  4.25612573e-01]]
[[ 7.99090782e+02  4.36498523e+02 -4.67111150e-02  2.63654187e-02]
 [ 7.97989168e+02  4.37991845e+02 -8.99766047e-02  2.60889398e-03]
 [ 7.99000723e+02  4.35708302e+02  4.04707009e-01  6.66692144e-01]
 ...
 [ 7.97320697e+02  4.36339957e+02 -1.51828023e-01  5.68549462e-01]
 [ 7.99327529e+02  4.36789571e+02  1.21560981e-01  1.41248045e-01]
 [ 7.99142936e+02  4.35109334e+02  2.00292401e-01  1.52732544e-01]]
[[ 7.97940850e+02  4.36861871e+02 -3.26014309e-01  4.72416893e-01]
 [ 7.98413313e+02  4.39499435e+02  2.38493534e-03  9.78634863e-01]
 [ 8.00465438e+02  4.34903369e+02  6.02484006e-01 

[[ 7.96772086e+02  4.29432345e+02 -4.88550276e-02 -1.92716700e-01]
 [ 7.96962834e+02  4.30923737e+02  7.86668304e-02 -5.28862549e-01]
 [ 7.96420189e+02  4.32121143e+02 -1.46296179e-01 -2.57967608e-01]
 ...
 [ 7.96782572e+02  4.32281008e+02  9.16132088e-02  4.46680784e-01]
 [ 7.96657496e+02  4.30895658e+02 -5.81042114e-01 -1.31876866e-01]
 [ 7.97147417e+02  4.30800557e+02 -2.39368196e-01  4.72688616e-02]]
[[ 7.97394797e+02  4.31050481e+02 -2.86954180e-01 -5.10488180e-01]
 [ 7.96655253e+02  4.29374532e+02 -2.44844657e-01 -1.69390452e-01]
 [ 7.96954485e+02  4.32930816e+02 -6.58363503e-02  7.74295504e-03]
 ...
 [ 7.94780485e+02  4.29304444e+02 -4.16845615e-01  1.76323652e-04]
 [ 7.96520730e+02  4.30389687e+02  1.11223686e-01 -1.20621524e-02]
 [ 7.95705043e+02  4.31489964e+02  3.39082322e-01  3.76655363e-01]]
[[ 7.97069549e+02  4.30071149e+02  4.56992980e-01 -1.09231479e-02]
 [ 7.95908955e+02  4.31290115e+02  8.17933607e-02 -1.57186734e-01]
 [ 7.95341645e+02  4.32028792e+02 -2.52815387e-01 

In [35]:
import cv2
import numpy as np

# Create a VideoCapture object
capture = cv2.VideoCapture('Trafic.mp4')

# Set up the desired color range (in this case, red)
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])

while True:
    # Read a frame from the video
    ret, frame = capture.read()

    # Check if the video has ended
    if not ret:
        break

    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create a mask that only includes pixels within the color range
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Apply the mask to the frame
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Show the result
    cv2.imshow('Result', result)
    print(result)

    # Check if the user has pressed the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
capture.release()

# Close all windows
cv2.destroyAllWindows()


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...


In [19]:
import cv2
import numpy as np

# Create a VideoCapture object
capture = cv2.VideoCapture('Trafic.mp4')

while True:
    # Read a frame from the video
    ret, frame = capture.read()

    # Check if the video has ended
    if not ret:
        break

    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the range of colors to detect
    lower_color = np.array([0, 0, 0])
    upper_color = np.array([255, 255, 255])

    # Create a mask that only includes pixels within the color range
    mask = cv2.inRange(hsv, lower_color, upper_color)

    # Calculate the color histogram of the mask
    histogram = cv2.calcHist([mask], [0], None, [256], [0, 256])

    # Find the index of the dominant color
    dominant_color_index = np.argmax(histogram)

    # Map the index to a color value
    dominant_color = dominant_color_index

    # Print the dominant color
    print(dominant_color)

    # Check if the user has pressed the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
capture.release()

# Close all windows
cv2.destroyAllWindows()


255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
